In [1]:
"""
This script applies the power local recession curve watertable fluctuation method (e.g., Wendland
et al. 2007) to estimate recharge from synthetic hydrographs. The method is implemented on a series of generated 
recharge events, and recharge estimates are compared to known values. The output is the relative error (%) of the 
recharge estimate, calculated as described in Becke et al. (2025).
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from pathlib import Path
import sys

# Print environment info
print(sys.version)
print(f'numpy version: {np.__version__}')

# Setup paths
FOLDER_NAME = "LRC_Power"
BASE_DIR = Path.home() / "workspace" / "WTF" / "OutputFiles" 
OUTPUT_DIR = BASE_DIR / FOLDER_NAME
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

INPUT_DIR = BASE_DIR / "data" 

# Define time series and event windows
time = np.linspace(0, 400., 401)
event_windows = {
    1: {"fit": (30, 90), "extend": (30, 120), "check": 120},
    2: {"fit": (120, 210), "extend": (120, 240), "check": 240},
    3: {"fit": (240, 360), "extend": (240, 390), "check": 390},
}

def power_law(x, C1, C2, C3):
    return C1 * np.power(x - C3, C2)

def compute_LRC_errors(itr, event_id, time, fit_range, extend_range, check_index):
    x_fit = time[fit_range[0]:fit_range[1] + 1]
    x_extend = time[extend_range[0]:extend_range[1] + 1]

    errors = []

    for j in range(1, itr + 1):
        try:
            wl_file = INPUT_DIR / f"hydro_{j:04d}.csv"
            var_file = INPUT_DIR / f"var_{j:04d}.csv"

            wl = pd.read_csv(wl_file, header=None).to_numpy().flatten()
            var = pd.read_csv(var_file, header=None).to_numpy()

            y_fit = wl[fit_range[0]:fit_range[1] + 1]

            # Fit power-law curve
            maxfev = 5000
            fitted = False
            while maxfev <= 1_000_000:
                try:
                    params, _ = curve_fit(power_law, x_fit, y_fit, absolute_sigma=True,
                                          maxfev=maxfev, method='trf')
                    C1, C2, C3 = params
                    fitted = True
                    break
                except (RuntimeError, ValueError):
                    print(f"[RE{event_id}] Hydrograph {j:04d}: Failed fit with maxfev={maxfev}, retrying...")
                    maxfev *= 10

            if not fitted:
                print(f"[RE{event_id}] Hydrograph {j:04d}: No fit found. Skipping.")
                continue

            y_proj = power_law(x_extend, C1, C2, C3)
            delta_h = wl[check_index] - y_proj[-1]
            delta_t = x_extend[-1] - x_fit[-1]

            Sy = var[3][0]
            true_rech = var[0][0]
            rech_est = (delta_h / delta_t) * Sy
            rech_error = ((rech_est - true_rech) / true_rech) * 100

            errors.append(rech_error)

            print(f"[RE{event_id}] File {j:04d} | y = {C1:.5e} * (x - {C3:.2f}) ^ {C2:.5f} | Error = {rech_error:.2f}%")

        except Exception as e:
            print(f"[RE{event_id}] Hydrograph {j:04d} | Error: {e}")
            continue

    return errors

# --- Run for all events ---
for event_id, params in event_windows.items():
    errors = compute_LRC_errors(
        itr=1000,
        event_id=event_id,
        time=time,
        fit_range=params["fit"],
        extend_range=params["extend"],
        check_index=params["check"]
    )

    output_file = OUTPUT_DIR / f"RE{event_id}_Power_Rerror.csv"
    np.savetxt(output_file, errors, delimiter=',', header=f'power_Rerror_Event{event_id}', comments='')
    print(f"Saved errors for Event {event_id} to: {output_file}")


3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]
numpy version: 1.24.3
[RE1] File 0001 | y = 1.41187e+00 * (x - -18.37) ^ -0.66434 | Error = -10.51%
[RE1] Hydrograph 0002: Failed fit with maxfev=5000, retrying...
[RE1] Hydrograph 0002: Failed fit with maxfev=50000, retrying...


C:\Program Files\Anaconda3\Lib\site-packages\scipy\optimize\_lsq\common.py:115: RuntimeWarning: divide by zero encountered in divide
  phi_prime = -np.sum(suf ** 2 / denom**3) / p_norm


[RE1] File 0002 | y = 2.34481e+52 * (x - -11318.13) ^ -12.86273 | Error = -4.14%
[RE1] Hydrograph 0003: Failed fit with maxfev=5000, retrying...


C:\Program Files\Anaconda3\Lib\site-packages\scipy\optimize\_lsq\common.py:115: RuntimeWarning: divide by zero encountered in divide
  phi_prime = -np.sum(suf ** 2 / denom**3) / p_norm


[RE1] File 0003 | y = 1.96257e+54 * (x - -1220.24) ^ -17.60769 | Error = -9.03%
[RE1] File 0004 | y = 1.93444e+00 * (x - 23.79) ^ -0.40904 | Error = -66.17%
[RE1] File 0005 | y = 5.43206e+00 * (x - 5.88) ^ -0.55842 | Error = -23.89%
[RE1] Hydrograph 0006: Failed fit with maxfev=5000, retrying...


...

[RE3] File 0008 | y = 1.49725e+54 * (x - -432.05) ^ -19.39927 | Error = -26.81%
[RE3] Hydrograph 0009: Failed fit with maxfev=5000, retrying...


C:\Program Files\Anaconda3\Lib\site-packages\scipy\optimize\_lsq\common.py:115: RuntimeWarning: divide by zero encountered in divide
  phi_prime = -np.sum(suf ** 2 / denom**3) / p_norm


[RE3] File 0009 | y = 1.68179e+54 * (x - -4060.82) ^ -14.74487 | Error = -5.18%
[RE3] Hydrograph 0010: Failed fit with maxfev=5000, retrying...


C:\Program Files\Anaconda3\Lib\site-packages\scipy\optimize\_lsq\common.py:115: RuntimeWarning: divide by zero encountered in divide
  phi_prime = -np.sum(suf ** 2 / denom**3) / p_norm


[RE3] File 0010 | y = 2.90946e+54 * (x - -3681.44) ^ -15.02684 | Error = -5.29%
Saved errors for Event 3 to: C:\Users\beck0213\workspace\WTF\OutputFiles\GITHUB_CHECK\LRC_Power\RE3_Power_Rerror.csv
